# 1) Importing libraries 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')

In [2]:
from pandasql import sqldf

In [3]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [4]:
from sklearn.model_selection import KFold

In [5]:
import dask.dataframe as dd

In [6]:
np.random.seed(51)

In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 2) Reading train_metadata_kaggle & test_metadata_kaggle

In [8]:
%%time
train_metadata_kaggle = dd.read_csv('train_metadata_kaggle.csv')
test_metadata_kaggle = dd.read_csv('test_metadata_kaggle.csv')
train_metadata_kaggle = train_metadata_kaggle.compute()
test_metadata_kaggle = test_metadata_kaggle.compute()
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

(7848, 51) (3492890, 50)
CPU times: user 1min 22s, sys: 4.84 s, total: 1min 27s
Wall time: 15.5 s


In [9]:
test_id = test_metadata_kaggle['object_id']

In [10]:
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

(7848, 51) (3492890, 50)


In [11]:
test_metadata_kaggle.head()

,object_id,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_diff,flux_dif2,flux_w_mean,flux_dif3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,mwebv
0,13,-12.680235,42.765503,3.997127,0.616561,9.149645,2.037355,0.691634,11.257108,2.461810,1.972973,1.718101,1.826388,0.157576,7806.412424,4.771625,1.896346e+05,5.396523,55.445738,13.871398,24.292155,2.282455,29.002872,37.684425,-0.247160,0.147622,92.039690,79.990817,10.567412,3.117684,164.640622,139.733762,4.512783,2.171455,300.547278,246.788411,2.951479,1.806797,408.305525,349.008202,0.094963,1.153076,324.515880,251.116361,0.072799,0.804929,119.8531,0.3193,0.0542,0.019
1,14,-11.142164,14.839427,0.884047,0.072856,3.399946,0.970525,0.690589,11.249375,2.457580,1.973559,1.717591,1.826703,0.012121,806.406927,11.486148,5.525817e+03,12.348124,25.981591,29.389389,6.852393,3.791608,22.708482,26.159787,0.099267,0.502325,13.685195,27.630359,13.429229,3.109318,31.012899,33.427074,13.769006,3.247873,56.042403,59.784625,1.750840,1.082798,83.561278,77.494564,-0.322108,-0.099957,84.725142,27.013154,-0.348744,0.132025,28.0053,0.6323,0.0179,0.018
2,17,-14.202744,16.761280,0.791032,0.458390,3.886578,0.377131,0.663680,11.278636,2.702947,2.184483,1.922641,1.802497,0.014205,784.835502,9.509911,4.124400e+03,9.923556,30.964024,39.143819,5.255113,5.892170,6.030862,46.450439,0.590786,-0.427429,53.366119,41.192678,4.127763,1.710048,58.607170,42.353288,8.642889,2.570936,26.975615,59.958041,3.484929,1.141959,57.222812,72.448018,2.085327,0.429582,76.240782,73.533783,0.063837,-0.175525,2.7628,0.8297,0.0605,0.016
3,23,-12.631923,28.061138,0.970396,0.465986,4.100713,2.177402,0.697639,11.305429,2.480364,2.003704,1.723367,1.821626,0.018182,876.027511,8.010597,8.293673e+03,9.227223,40.693061,41.934474,9.467365,4.298245,0.776942,14.926490,-0.580529,-0.058688,2.371129,22.476077,1.429683,-0.000571,34.447730,14.240274,1.223998,0.468594,58.911457,62.709899,4.269139,1.414380,115.918277,70.371095,7.377841,2.409871,112.547521,123.413344,1.793238,0.953107,3.0125,0.6533,0.1479,0.023
4,34,-13.239577,124.475609,4.580070,0.301366,19.862714,4.537527,0.679312,11.365292,2.746784,2.210577,1.930790,1.798629,0.065341,47612.580669,9.140568,4.815012e+06,10.251332,137.715186,30.068359,101.128982,1.361778,65.064185,55.191226,7.457425,2.135457,202.641918,191.410916,16.393710,4.133929,425.940451,414.455381,15.206929,4.036551,368.228927,360.630612,15.386561,4.030676,377.744797,301.983950,15.302549,3.992740,172.564336,289.757599,12.507289,3.513977,25.0572,0.4617,0.0122,0.023


# 3) reading train_metadata_final & test_metadata_final

In [ ]:
%%time
train_metadata = dd.read_csv('train_metadata_final.csv')
test_metadata = dd.read_csv('test_metadata_final.csv')
train_metadata = train_metadata.compute()
test_metadata = test_metadata.compute()
print(train_metadata.shape,test_metadata.shape)

In [ ]:
y = train_metadata['target']
del train_metadata['target']
gc.collect()

# 4) Reading created_column

In [12]:
created_column = pd.read_csv('hostgal_specz.csv',header=None)

In [13]:
created_column.head()

,0,1
0,615,0.0000
1,713,1.8180
2,730,0.2320
3,745,0.3037
4,1124,0.1934


In [14]:
created_column.shape[0] == train_metadata_kaggle.shape[0] + test_metadata_kaggle.shape[0]

True

In [15]:
created_column.columns = ['object_id','hostgal_specz']

In [16]:
print(train_metadata_kaggle.shape)

(7848, 51)


# 5) Removing duplicated_columns from train_metadata & test_metadata

In [17]:
will_be_deleted_columns = ['NG_min_flux','NG_max_flux','NG_std_flux','NG_mean_flux','NG_median_flux',
                          'NG_min_flux_err','NG_max_flux_err','NG_std_flux_err','NG_mean_flux_err','NG_median_flux_err',
                          'NG_mean_detected','hostgal_photoz','hostgal_photoz_err','mwebv']

In [18]:
"""%%time
train_metadata.drop(will_be_deleted_columns,inplace=True , axis=1)
test_metadata.drop(will_be_deleted_columns,inplace=True , axis=1)
gc.collect()"""

'%%time\ntrain_metadata.drop(will_be_deleted_columns,inplace=True , axis=1)\ntest_metadata.drop(will_be_deleted_columns,inplace=True , axis=1)\ngc.collect()'

In [20]:
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

(7848, 51) (3492890, 50)


In [22]:
train_metadata_kaggle.head()

,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_diff,flux_dif2,flux_w_mean,flux_dif3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,mwebv,target,object_id
0,-1100.440063,660.626343,-123.096998,-89.477524,394.109851,-0.349540,2.130510,12.845472,4.482743,3.835268,1.744747,1.623740,0.946023,2.929669e+06,0.812722,-9.601766e+08,-1.414322,1761.066406,-14.306331,-327.742307,-5.373326,205.036926,1628.427737,-1.475181,0.128917,22370.594834,2806.374162,-1.255123,0.415580,7780.500807,2805.598113,-1.409885,0.339918,7024.003068,2536.068846,-1.449858,0.293128,3245.366349,2741.539785,-1.548319,0.200096,2704.641265,2893.344217,-1.592820,0.125268,873.7903,0.0000,0.0000,0.017,92,615
1,-14.735178,14.770886,-1.423351,-0.873033,6.471144,0.014989,0.639458,9.115748,2.359620,1.998217,1.509888,1.633246,0.171429,5.886068e+03,3.439423,-2.875087e+04,-3.454554,29.506064,-20.730002,-4.884564,-6.040676,190.427851,299.586559,-1.014003,0.260052,57.109047,192.539229,-1.097170,-0.087865,44.477327,191.057528,-1.188472,-0.022678,55.270113,212.522263,-1.142896,-0.167176,50.414646,203.892482,-1.190245,-0.064134,100.473776,143.963093,-0.797047,0.218182,846.8017,1.6267,0.2552,0.007,88,713
2,-19.159811,47.310059,2.267434,0.409172,8.022239,3.177854,0.695106,11.281384,2.471061,1.990851,1.721134,1.823726,0.069697,4.124452e+03,5.480405,1.046502e+05,5.989138,66.469870,29.315018,25.373110,2.619697,3.461790,4.729538,0.474215,0.356910,7.334944,13.515895,0.976374,0.471342,124.845250,119.500254,5.131290,2.385066,168.280524,162.799417,7.125665,2.662075,219.745132,202.532898,6.081065,2.537802,231.509177,199.286370,3.583130,1.680352,78.7737,0.2262,0.0157,0.021,42,730
3,-15.494463,220.795212,8.909206,1.035895,27.558208,4.979826,0.567170,55.892746,2.555576,1.819875,3.537324,10.741655,0.173789,9.416165e+04,9.611274,1.439125e+07,11.141069,236.289675,26.521968,152.835617,1.546038,129.421659,123.298327,4.629801,2.023211,320.174052,280.440312,50.868880,7.007099,543.845781,491.548270,36.088137,5.688194,807.123762,710.721942,16.392533,3.751603,735.528417,680.055280,13.747434,3.476420,591.037583,523.503586,12.134629,3.170857,123.6872,0.2813,1.1523,0.007,90,745
4,-16.543753,143.600189,7.145702,1.141288,20.051722,4.406298,0.695277,11.383690,2.753004,2.214854,1.933837,1.794938,0.173295,3.432418e+04,7.868462,3.015599e+06,7.908174,160.143942,22.411225,87.856390,1.822792,41.639721,32.987125,0.822496,-0.332169,268.808929,207.812015,6.112295,2.377222,594.150153,498.509820,10.343254,3.075437,643.020183,555.512641,14.095862,3.603208,574.553907,524.107264,16.377058,3.904008,393.114268,357.907185,14.434470,3.657305,133.9113,0.2415,0.0176,0.024,90,1124


In [ ]:
#for i,j in enumerate(list(train_metadata.columns.values)):
#    print(i,j)

# 6) Adding hostgal_specz to train_metadata_kaggle

In [23]:
%%time
train_metadata_kaggle = train_metadata_kaggle.merge(right = created_column ,how='left',on = 'object_id')

In [24]:
train_metadata_kaggle.head()

,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_diff,flux_dif2,flux_w_mean,flux_dif3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,mwebv,target,object_id,hostgal_specz
0,-1100.440063,660.626343,-123.096998,-89.477524,394.109851,-0.349540,2.130510,12.845472,4.482743,3.835268,1.744747,1.623740,0.946023,2.929669e+06,0.812722,-9.601766e+08,-1.414322,1761.066406,-14.306331,-327.742307,-5.373326,205.036926,1628.427737,-1.475181,0.128917,22370.594834,2806.374162,-1.255123,0.415580,7780.500807,2805.598113,-1.409885,0.339918,7024.003068,2536.068846,-1.449858,0.293128,3245.366349,2741.539785,-1.548319,0.200096,2704.641265,2893.344217,-1.592820,0.125268,873.7903,0.0000,0.0000,0.017,92,615,0.0000
1,-14.735178,14.770886,-1.423351,-0.873033,6.471144,0.014989,0.639458,9.115748,2.359620,1.998217,1.509888,1.633246,0.171429,5.886068e+03,3.439423,-2.875087e+04,-3.454554,29.506064,-20.730002,-4.884564,-6.040676,190.427851,299.586559,-1.014003,0.260052,57.109047,192.539229,-1.097170,-0.087865,44.477327,191.057528,-1.188472,-0.022678,55.270113,212.522263,-1.142896,-0.167176,50.414646,203.892482,-1.190245,-0.064134,100.473776,143.963093,-0.797047,0.218182,846.8017,1.6267,0.2552,0.007,88,713,1.8180
2,-19.159811,47.310059,2.267434,0.409172,8.022239,3.177854,0.695106,11.281384,2.471061,1.990851,1.721134,1.823726,0.069697,4.124452e+03,5.480405,1.046502e+05,5.989138,66.469870,29.315018,25.373110,2.619697,3.461790,4.729538,0.474215,0.356910,7.334944,13.515895,0.976374,0.471342,124.845250,119.500254,5.131290,2.385066,168.280524,162.799417,7.125665,2.662075,219.745132,202.532898,6.081065,2.537802,231.509177,199.286370,3.583130,1.680352,78.7737,0.2262,0.0157,0.021,42,730,0.2320
3,-15.494463,220.795212,8.909206,1.035895,27.558208,4.979826,0.567170,55.892746,2.555576,1.819875,3.537324,10.741655,0.173789,9.416165e+04,9.611274,1.439125e+07,11.141069,236.289675,26.521968,152.835617,1.546038,129.421659,123.298327,4.629801,2.023211,320.174052,280.440312,50.868880,7.007099,543.845781,491.548270,36.088137,5.688194,807.123762,710.721942,16.392533,3.751603,735.528417,680.055280,13.747434,3.476420,591.037583,523.503586,12.134629,3.170857,123.6872,0.2813,1.1523,0.007,90,745,0.3037
4,-16.543753,143.600189,7.145702,1.141288,20.051722,4.406298,0.695277,11.383690,2.753004,2.214854,1.933837,1.794938,0.173295,3.432418e+04,7.868462,3.015599e+06,7.908174,160.143942,22.411225,87.856390,1.822792,41.639721,32.987125,0.822496,-0.332169,268.808929,207.812015,6.112295,2.377222,594.150153,498.509820,10.343254,3.075437,643.020183,555.512641,14.095862,3.603208,574.553907,524.107264,16.377058,3.904008,393.114268,357.907185,14.434470,3.657305,133.9113,0.2415,0.0176,0.024,90,1124,0.1934


# 7) Adding hostgal_specz to train_metadata_kaggle

In [25]:
%%time
test_metadata_kaggle = test_metadata_kaggle.merge(right = created_column ,how='left',on = 'object_id')

CPU times: user 1.37 s, sys: 604 ms, total: 1.98 s
Wall time: 1.97 s


In [26]:
test_metadata_kaggle.head()

,object_id,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_diff,flux_dif2,flux_w_mean,flux_dif3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,mwebv,hostgal_specz
0,13,-12.680235,42.765503,3.997127,0.616561,9.149645,2.037355,0.691634,11.257108,2.461810,1.972973,1.718101,1.826388,0.157576,7806.412424,4.771625,1.896346e+05,5.396523,55.445738,13.871398,24.292155,2.282455,29.002872,37.684425,-0.247160,0.147622,92.039690,79.990817,10.567412,3.117684,164.640622,139.733762,4.512783,2.171455,300.547278,246.788411,2.951479,1.806797,408.305525,349.008202,0.094963,1.153076,324.515880,251.116361,0.072799,0.804929,119.8531,0.3193,0.0542,0.019,0.304700
1,14,-11.142164,14.839427,0.884047,0.072856,3.399946,0.970525,0.690589,11.249375,2.457580,1.973559,1.717591,1.826703,0.012121,806.406927,11.486148,5.525817e+03,12.348124,25.981591,29.389389,6.852393,3.791608,22.708482,26.159787,0.099267,0.502325,13.685195,27.630359,13.429229,3.109318,31.012899,33.427074,13.769006,3.247873,56.042403,59.784625,1.750840,1.082798,83.561278,77.494564,-0.322108,-0.099957,84.725142,27.013154,-0.348744,0.132025,28.0053,0.6323,0.0179,0.018,0.636228
2,17,-14.202744,16.761280,0.791032,0.458390,3.886578,0.377131,0.663680,11.278636,2.702947,2.184483,1.922641,1.802497,0.014205,784.835502,9.509911,4.124400e+03,9.923556,30.964024,39.143819,5.255113,5.892170,6.030862,46.450439,0.590786,-0.427429,53.366119,41.192678,4.127763,1.710048,58.607170,42.353288,8.642889,2.570936,26.975615,59.958041,3.484929,1.141959,57.222812,72.448018,2.085327,0.429582,76.240782,73.533783,0.063837,-0.175525,2.7628,0.8297,0.0605,0.016,0.818608
3,23,-12.631923,28.061138,0.970396,0.465986,4.100713,2.177402,0.697639,11.305429,2.480364,2.003704,1.723367,1.821626,0.018182,876.027511,8.010597,8.293673e+03,9.227223,40.693061,41.934474,9.467365,4.298245,0.776942,14.926490,-0.580529,-0.058688,2.371129,22.476077,1.429683,-0.000571,34.447730,14.240274,1.223998,0.468594,58.911457,62.709899,4.269139,1.414380,115.918277,70.371095,7.377841,2.409871,112.547521,123.413344,1.793238,0.953107,3.0125,0.6533,0.1479,0.023,0.580034
4,34,-13.239577,124.475609,4.580070,0.301366,19.862714,4.537527,0.679312,11.365292,2.746784,2.210577,1.930790,1.798629,0.065341,47612.580669,9.140568,4.815012e+06,10.251332,137.715186,30.068359,101.128982,1.361778,65.064185,55.191226,7.457425,2.135457,202.641918,191.410916,16.393710,4.133929,425.940451,414.455381,15.206929,4.036551,368.228927,360.630612,15.386561,4.030676,377.744797,301.983950,15.302549,3.992740,172.564336,289.757599,12.507289,3.513977,25.0572,0.4617,0.0122,0.023,0.455800


In [27]:
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

(7848, 52) (3492890, 51)


# 8) Creating train and test datasets

In [28]:
ignored_columns = ['ra','decl','gal_l','gal_b','ddf','hostgal_specz','distmod']

In [29]:
#temp = train_metadata_kaggle.merge(right = train_metadata[[x for x in train_metadata.columns if x not in ignored_columns ]] ,how='left',on = 'object_id')

In [30]:
temp = train_metadata_kaggle.copy()

In [31]:
y = temp['target']

In [32]:
del temp['target']

In [33]:
print(temp.shape)

(7848, 51)


In [34]:
temp.head()

,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_diff,flux_dif2,flux_w_mean,flux_dif3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,mwebv,object_id,hostgal_specz
0,-1100.440063,660.626343,-123.096998,-89.477524,394.109851,-0.349540,2.130510,12.845472,4.482743,3.835268,1.744747,1.623740,0.946023,2.929669e+06,0.812722,-9.601766e+08,-1.414322,1761.066406,-14.306331,-327.742307,-5.373326,205.036926,1628.427737,-1.475181,0.128917,22370.594834,2806.374162,-1.255123,0.415580,7780.500807,2805.598113,-1.409885,0.339918,7024.003068,2536.068846,-1.449858,0.293128,3245.366349,2741.539785,-1.548319,0.200096,2704.641265,2893.344217,-1.592820,0.125268,873.7903,0.0000,0.0000,0.017,615,0.0000
1,-14.735178,14.770886,-1.423351,-0.873033,6.471144,0.014989,0.639458,9.115748,2.359620,1.998217,1.509888,1.633246,0.171429,5.886068e+03,3.439423,-2.875087e+04,-3.454554,29.506064,-20.730002,-4.884564,-6.040676,190.427851,299.586559,-1.014003,0.260052,57.109047,192.539229,-1.097170,-0.087865,44.477327,191.057528,-1.188472,-0.022678,55.270113,212.522263,-1.142896,-0.167176,50.414646,203.892482,-1.190245,-0.064134,100.473776,143.963093,-0.797047,0.218182,846.8017,1.6267,0.2552,0.007,713,1.8180
2,-19.159811,47.310059,2.267434,0.409172,8.022239,3.177854,0.695106,11.281384,2.471061,1.990851,1.721134,1.823726,0.069697,4.124452e+03,5.480405,1.046502e+05,5.989138,66.469870,29.315018,25.373110,2.619697,3.461790,4.729538,0.474215,0.356910,7.334944,13.515895,0.976374,0.471342,124.845250,119.500254,5.131290,2.385066,168.280524,162.799417,7.125665,2.662075,219.745132,202.532898,6.081065,2.537802,231.509177,199.286370,3.583130,1.680352,78.7737,0.2262,0.0157,0.021,730,0.2320
3,-15.494463,220.795212,8.909206,1.035895,27.558208,4.979826,0.567170,55.892746,2.555576,1.819875,3.537324,10.741655,0.173789,9.416165e+04,9.611274,1.439125e+07,11.141069,236.289675,26.521968,152.835617,1.546038,129.421659,123.298327,4.629801,2.023211,320.174052,280.440312,50.868880,7.007099,543.845781,491.548270,36.088137,5.688194,807.123762,710.721942,16.392533,3.751603,735.528417,680.055280,13.747434,3.476420,591.037583,523.503586,12.134629,3.170857,123.6872,0.2813,1.1523,0.007,745,0.3037
4,-16.543753,143.600189,7.145702,1.141288,20.051722,4.406298,0.695277,11.383690,2.753004,2.214854,1.933837,1.794938,0.173295,3.432418e+04,7.868462,3.015599e+06,7.908174,160.143942,22.411225,87.856390,1.822792,41.639721,32.987125,0.822496,-0.332169,268.808929,207.812015,6.112295,2.377222,594.150153,498.509820,10.343254,3.075437,643.020183,555.512641,14.095862,3.603208,574.553907,524.107264,16.377058,3.904008,393.114268,357.907185,14.434470,3.657305,133.9113,0.2415,0.0176,0.024,1124,0.1934


# 9) Writing helpful functions

In [35]:
classes = sorted(y.unique())

# Taken from Giba's topic : https://www.kaggle.com/titericz
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
# with Kyle Boone's post https://www.kaggle.com/kyleboone
class_weight = {
    c: 1 for c in classes
}
for c in [64, 15]:
    class_weight[c] = 2

print('Unique classes : ', classes)

train_id = temp['object_id']
del temp['object_id']

Unique classes :  [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]


In [36]:
# Compute weights
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}

def multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds
    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return loss


def lgb_multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds.reshape(y_true.shape[0], len(classes), order='F')

    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return 'wloss', loss, False

# 10 ) Defining hyperparameters

In [37]:
%%time
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
clfs = []
importances = pd.DataFrame()
lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 14,
    'metric': 'multi_logloss',
    'learning_rate': 0.03,
    'subsample': .9,
    'colsample_bytree': 0.5,
    'reg_alpha': .01,
    'reg_lambda': .01,
    'min_split_gain': 0.01,
    'min_child_weight': 10,
    'n_estimators': 1000,
    'silent': -1,
    'verbose': -1,
    'max_depth': 3
}

CPU times: user 318 µs, sys: 21 µs, total: 339 µs
Wall time: 344 µs


In [38]:
# Compute weights
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}

oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))

# 11) Training

In [39]:
%%time
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
    val_x, val_y = temp.iloc[val_], y.iloc[val_]

    clf = lgb.LGBMClassifier(**lgb_params)
    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        eval_metric=lgb_multi_weighted_logloss,
        verbose=100,
        early_stopping_rounds=50,
        sample_weight=trn_y.map(weights)
    )
    oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
    print(multi_weighted_logloss(val_y, oof_preds[val_, :]))

    imp_df = pd.DataFrame()
    imp_df['feature'] = temp.columns
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)

    clfs.append(clf)

print('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))

Training until validation scores don't improve for 50 rounds.
[100]	training's multi_logloss: 0.778804	training's wloss: 0.769746	valid_1's multi_logloss: 1.11383	valid_1's wloss: 0.920765
[200]	training's multi_logloss: 0.523447	training's wloss: 0.511953	valid_1's multi_logloss: 0.888801	valid_1's wloss: 0.731165
[300]	training's multi_logloss: 0.412603	training's wloss: 0.40083	valid_1's multi_logloss: 0.801819	valid_1's wloss: 0.68215
[400]	training's multi_logloss: 0.344029	training's wloss: 0.332515	valid_1's multi_logloss: 0.754687	valid_1's wloss: 0.664758
[500]	training's multi_logloss: 0.295014	training's wloss: 0.28418	valid_1's multi_logloss: 0.721985	valid_1's wloss: 0.660009
Early stopping, best iteration is:
[503]	training's multi_logloss: 0.293798	training's wloss: 0.282967	valid_1's multi_logloss: 0.721182	valid_1's wloss: 0.659682
0.6596821500144557
Training until validation scores don't improve for 50 rounds.
[100]	training's multi_logloss: 0.786365	training's wloss:

In [40]:
print(list(temp.columns.values)[:5])

['flux_min', 'flux_max', 'flux_mean', 'flux_median', 'flux_std']


In [41]:
print(list(test_metadata_kaggle.columns.values)[:5])

['object_id', 'flux_min', 'flux_max', 'flux_mean', 'flux_median']


In [ ]:
imp_df.sort_values(by ='gain',ascending=False).reset_index(drop=True).head(20)

# 12) Preparing test data for scoring

In [ ]:
%%time
#temp_test = test_metadata_kaggle.merge(right = test_metadata[[x for x in test_metadata.columns if x not in ignored_columns ]] ,how='left',on = 'object_id')

In [42]:
%%time
temp_test = test_metadata_kaggle.copy()

CPU times: user 798 ms, sys: 1.24 s, total: 2.04 s
Wall time: 2.04 s


In [43]:
del temp_test['object_id']

In [44]:
print(temp.shape,temp_test.shape)

(7848, 50) (3492890, 50)


In [45]:
list(temp.columns) == list(temp_test.columns)

True

In [46]:
gc.enable()
del train_metadata_kaggle,test_metadata_kaggle
gc.collect()

306

In [47]:
%%time
test_pred0 = pd.DataFrame()
test_pred1 = pd.DataFrame()
test_pred2 = pd.DataFrame()
test_pred3 = pd.DataFrame()
test_pred4 = pd.DataFrame()

list_of_df = [test_pred0,test_pred1,test_pred2,test_pred3,test_pred4]

for num,c in enumerate(clfs):
    print(num)
    for k in range(0,len(temp_test),500000):
        print(k)
        test_pred = pd.DataFrame(c.predict_proba(temp_test[ k:k+500000] ))
        list_of_df[num] = pd.concat([list_of_df[num],test_pred],axis=0)
        del test_pred

0
0
500000
1000000
1500000
2000000
2500000
3000000
1
0
500000
1000000
1500000
2000000
2500000
3000000
2
0
500000
1000000
1500000
2000000
2500000
3000000
3
0
500000
1000000
1500000
2000000
2500000
3000000
4
0
500000
1000000
1500000
2000000
2500000
3000000
CPU times: user 2h 3min 45s, sys: 14.1 s, total: 2h 4min
Wall time: 16min 25s


In [65]:
test_pred2 = pd.DataFrame()
test_pred2 = (list_of_df[0] + list_of_df[1] + list_of_df[2] + list_of_df[3] + list_of_df[4])/5

In [66]:
test_pred2.shape

(3492890, 14)

In [67]:
test_pred2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.000154,0.001983,0.000131,0.394665,0.335427,0.000140,0.074189,0.000119,0.000174,0.002804,0.000337,0.188987,0.000225,0.000665
1,0.000186,0.012466,0.000605,0.167946,0.040383,0.000191,0.036180,0.000776,0.000569,0.011499,0.004435,0.717055,0.000266,0.007443
2,0.000113,0.006480,0.000417,0.056533,0.022900,0.000115,0.014811,0.001228,0.000263,0.021086,0.013104,0.856241,0.000101,0.006609
3,0.000158,0.003694,0.000228,0.074904,0.012251,0.000147,0.131169,0.001514,0.000311,0.062992,0.001028,0.684085,0.000133,0.027386
4,0.000083,0.003798,0.000070,0.093691,0.120988,0.000074,0.015809,0.000080,0.000129,0.005454,0.000089,0.759022,0.000066,0.000647


In [68]:
test_pred2[14] = 0.19

In [69]:
test_pred2 = test_pred2 / 1.19

In [73]:
test_pred2 = test_pred2.reset_index(drop=True)

In [74]:
temp_columns = ['object_id','class_6','class_15','class_16','class_42','class_52','class_53','class_62','class_64','class_65','class_67','class_88','class_90','class_92','class_95','class_99']

In [75]:
test_pred2.columns = temp_columns[1:]

In [76]:
print(test_pred2.shape,test_id.shape)

(3492890, 15) (3492890,)


In [77]:
test_id.tail()

3492885    130787966
3492886    130787971
3492887    130787974
3492888    130788053
3492889    130788054
Name: object_id, dtype: int64

In [78]:
test_id = test_id.reset_index(drop=True)

In [80]:
test_id.index == test_pred2.index

array([ True,  True,  True, ...,  True,  True,  True])

In [82]:
%%time
test_pred = pd.concat([test_id,test_pred2],axis=1)

CPU times: user 98.5 ms, sys: 136 ms, total: 234 ms
Wall time: 232 ms


In [83]:
test_pred = test_pred[temp_columns]

In [84]:
test_pred.head()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
0,13,0.000129,0.001666,0.000110,0.331652,0.281872,0.000117,0.062344,0.000100,0.000147,0.002356,0.000283,0.158812,0.000189,0.000558,0.159664
1,14,0.000156,0.010476,0.000508,0.141131,0.033936,0.000161,0.030403,0.000652,0.000478,0.009663,0.003727,0.602567,0.000223,0.006255,0.159664
2,17,0.000095,0.005445,0.000350,0.047507,0.019244,0.000097,0.012446,0.001032,0.000221,0.017720,0.011012,0.719530,0.000085,0.005554,0.159664
3,23,0.000133,0.003104,0.000192,0.062945,0.010295,0.000124,0.110226,0.001272,0.000262,0.052934,0.000864,0.574861,0.000112,0.023013,0.159664
4,34,0.000070,0.003192,0.000059,0.078732,0.101671,0.000062,0.013285,0.000067,0.000108,0.004583,0.000075,0.637834,0.000055,0.000543,0.159664


In [85]:
test_pred.shape

(3492890, 16)

In [86]:
%%time
test_pred.to_csv('test_pred_6.csv',index=False)

CPU times: user 1min 33s, sys: 1.56 s, total: 1min 35s
Wall time: 1min 35s
